# 6. Interação

_“Um gráfico não é ‘desenhado’ de uma vez por todas; ele é ‘construído’ e reconstruído até revelar todas as relações constituídas pela interação dos dados. As melhores operações gráficas são aquelas realizadas pelo próprio tomador de decisões.”_ &mdash; [Jacques Bertin](https://books.google.com/books?id=csqX_xnm4tcC)

A visualização oferece um meio poderoso para compreender dados. No entanto, uma única imagem geralmente responde, no máximo, a algumas poucas perguntas. Por meio da _interação_, podemos transformar imagens estáticas em ferramentas de exploração: destacando pontos de interesse, ampliando para revelar padrões mais detalhados e estabelecendo conexões entre múltiplas visualizações para compreender relações multidimensionais.

No centro da interação está a noção de _selection_ (seleção): um meio de indicar ao computador quais elementos ou regiões nos interessam. Por exemplo, podemos passar o mouse sobre um ponto, clicar em múltiplas marcas ou desenhar uma caixa delimitadora ao redor de uma região para destacar subconjuntos dos dados para inspecionar melhor.

Além das codificações visuais e transformações de dados, Altair fornece uma abstração de _selection_ para a criação de interações. Essas seleções abrangem três aspectos:

1. Tratamento de eventos de entrada para selecionar pontos ou regiões de interesse, como passar o mouse, clicar, arrastar, rolar e eventos de toque.
2. Generalização a partir da entrada para formar uma regra de seleção (ou [_predicado_](https://en.wikipedia.org/wiki/Predicate_%28mathematical_logic%29)) que determina se um determinado registro de dados faz parte da seleção.
3. Utilização do predicado de seleção para configurar dinamicamente uma visualização, controlando _conditional encodings_ (codificações condicionais), _filter transforms_ (transformações de filtro) ou _scale domains_ (domínios de escala).


Este notebook introduz seleções interativas e explora como usá-las para criar uma variedade de técnicas de interação, como consultas dinâmicas, arrastamento &amp; zoom, detalhes sob demanda e seleção interativa &amp; _linking_.

_Este notebook é parte do [currículo de visualização de dados](https://github.com/uwdata/visualization-curriculum)._

In [1]:
import pandas as pd
import altair as alt

## Datasets

We will visualize a variety of datasets from the [vega-datasets](https://github.com/vega/vega-datasets) collection:

- A dataset of `cars` from the 1970s and early 1980s,
- A dataset of `movies`, previously used in the [Data Transformation](https://github.com/uwdata/visualization-curriculum/blob/master/altair_data_transformation.ipynb) notebook,
- A dataset containing ten years of [S&amp;P 500](https://en.wikipedia.org/wiki/S%26P_500_Index) (`sp500`) stock prices,
- A dataset of technology company `stocks`, and
- A dataset of `flights`, including departure time, distance, and arrival delay.

In [2]:
cars = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/cars.json'
movies = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/movies.json'
sp500 = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/sp500.csv'
stocks = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/stocks.csv'
flights = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/flights-5k.json'

## Introducing Selections

Let's start with a basic selection: simply clicking a point to highlight it. Using the `cars` dataset, we'll start with a scatter plot of horsepower versus miles per gallon, with a color encoding for the number cylinders in the car engine.

In addition, we'll create a selection instance by calling `alt.selection_single()`, indicating we want a selection defined over a _single value_. By default, the selection uses a mouse click to determine the selected value. To register a selection with a chart, we must add it using the `.add_selection()` method.

Once our selection has been defined, we can use it as a parameter for _conditional encodings_, which apply a different encoding depending on whether a data record lies in or out of the selection. For example, consider the following code:

~~~ python
color=alt.condition(selection, 'Cylinders:O', alt.value('grey'))
~~~

This encoding definition states that data points contained within the `selection` should be colored according to the `Cylinder` field, while non-selected data points should use a default `grey`. An empty selection includes _all_ data points, and so initially all points will be colored.

_Try clicking different points in the chart below. What happens? (Click the background to clear the selection state and return to an "empty" selection.)_

In [3]:
selection = alt.selection_single();
  
alt.Chart(cars).mark_circle().add_selection(
    selection
).encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.condition(selection, 'Cylinders:O', alt.value('grey')),
    opacity=alt.condition(selection, alt.value(0.8), alt.value(0.1))
)

alt.Chart(...)

Of course, highlighting individual data points one-at-a-time is not particularly exciting! As we'll see, however, single value selections provide a useful building block for more powerful interactions. Moreover, single value selections are just one of the three selection types provided by Altair:

- `selection_single` - select a single discrete value, by default on click events. 
- `selection_multi` - select multiple discrete values. The first value is selected on mouse click and additional values toggled using shift-click. 
- `selection_interval` - select a continuous range of values, initiated by mouse drag.

Let's compare each of these selection types side-by-side. To keep our code tidy we'll first define a function (`plot`) that generates a scatter plot specification just like the one above. We can pass a selection to the `plot` function to have it applied to the chart:

In [4]:
def plot(selection):
    return alt.Chart(cars).mark_circle().add_selection(
        selection
    ).encode(
        x='Horsepower:Q',
        y='Miles_per_Gallon:Q',
        color=alt.condition(selection, 'Cylinders:O', alt.value('grey')),
        opacity=alt.condition(selection, alt.value(0.8), alt.value(0.1))
    ).properties(
        width=240,
        height=180
    )

Let's use our `plot` function to create three chart variants, one per selection type.

The first (`single`) chart replicates our earlier example. The second (`multi`) chart supports shift-click interactions to toggle inclusion of multiple points within the selection. The third (`interval`) chart generates a selection region (or _brush_) upon mouse drag. Once created, you can drag the brush around to select different points, or scroll when the cursor is inside the brush to scale (zoom) the brush size.

_Try interacting with each of the charts below!_

In [5]:
alt.hconcat(
  plot(alt.selection_single()).properties(title='Single (Click)'),
  plot(alt.selection_multi()).properties(title='Multi (Shift-Click)'),
  plot(alt.selection_interval()).properties(title='Interval (Drag)')
)

alt.HConcatChart(...)

The examples above use default interactions (click, shift-click, drag) for each selection type. We can further customize the interactions by providing input event specifications using [Vega event selector syntax](https://vega.github.io/vega/docs/event-streams/). For example, we can modify our `single` and `multi` charts to trigger upon `mouseover` events instead of `click` events.

_Hold down the shift key in the second chart to "paint" with data!_

In [6]:
alt.hconcat(
  plot(alt.selection_single(on='mouseover')).properties(title='Single (Mouseover)'),
  plot(alt.selection_multi(on='mouseover')).properties(title='Multi (Shift-Mouseover)')
)

alt.HConcatChart(...)

Now that we've covered the basics of Altair selections, let's take a tour through the various interaction techniques they enable!

## Dynamic Queries

_Dynamic queries_ enables rapid, reversible exploration of data to isolate patterns of interest. As defined by [Ahlberg, Williamson, &amp; Shneiderman](https://www.cs.umd.edu/~ben/papers/Ahlberg1992Dynamic.pdf), a dynamic query:

- represents a query graphically,
- provides visible limits on the query range,
- provides a graphical representation of the data and query result,
- gives immediate feedback of the result after every query adjustment,
- and allows novice users to begin working with little training.

A common approach is to manipulate query parameters using standard user interface widgets such as sliders, radio buttons, and drop-down menus. To generate dynamic query widgets, we can apply a selection's `bind` operation to one or more data fields we wish to query.

Let's build an interactive scatter plot that uses a dynamic query to filter the display. Given a scatter plot of movie ratings (from Rotten Tomates and IMDB), we can add a selection over the `Major_Genre` field to enable interactive filtering by film genre.

To start, let's extract the unique (non-null) genres from the `movies` data:

In [7]:
df = pd.read_json(movies) # load movies data
genres = df['Major_Genre'].unique() # get unique field values
genres = list(filter(lambda d: d is not None, genres)) # filter out None values
genres.sort() # sort alphabetically

For later use, let's also define a list of unique `MPAA_Rating` values:

In [8]:
mpaa = ['G', 'PG', 'PG-13', 'R', 'NC-17', 'Not Rated']

Now let's create a `single` selection bound to a drop-down menu.

*Use the dynamic query menu below to explore the data. How do ratings vary by genre? How would you revise the code to filter `MPAA_Rating` (G, PG, PG-13, etc.) instead of `Major_Genre`?*

In [9]:
selectGenre = alt.selection_single(
    name='Select', # name the selection 'Select'
    fields=['Major_Genre'], # limit selection to the Major_Genre field
    init={'Major_Genre': genres[0]}, # use first genre entry as initial value
    bind=alt.binding_select(options=genres) # bind to a menu of unique genre values
)

alt.Chart(movies).mark_circle().add_selection(
    selectGenre
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(selectGenre, alt.value(0.75), alt.value(0.05))
)

alt.Chart(...)

Our construction above leverages multiple aspects of selections:

- We give the selection a name (`'Select'`). This name is not required, but allows us to influence the label text of the generated dynamic query menu. (_What happens if you remove the name? Try it!_)
- We constrain the selection to a specific data field (`Major_Genre`). Earlier when we used a `single` selection, the selection mapped to individual data points. By limiting the selection to a specific field, we can select _all_ data points whose `Major_Genre` field value matches the single selected value.
- We initialize `init=...` the selection to a starting value.
- We `bind` the selection to an interface widget, in this case a drop-down menu via `binding_select`.
- As before, we then use a conditional encoding to control the opacity channel.

### Binding Selections to Multiple Inputs

One selection instance can be bound to _multiple_ dynamic query widgets. Let's modify the example above to provide filters for _both_ `Major_Genre` and `MPAA_Rating`, using radio buttons instead of a menu. Our `single` selection is now defined over a single _pair_ of genre and MPAA rating values

_Look for surprising conjunctions of genre and rating. Are there any G or PG-rated horror films?_

In [10]:
# single-value selection over [Major_Genre, MPAA_Rating] pairs
# use specific hard-wired values as the initial selected values
selection = alt.selection_single(
    name='Select',
    fields=['Major_Genre', 'MPAA_Rating'],
    init={'Major_Genre': 'Drama', 'MPAA_Rating': 'R'},
    bind={'Major_Genre': alt.binding_select(options=genres), 'MPAA_Rating': alt.binding_radio(options=mpaa)}
)
  
# scatter plot, modify opacity based on selection
alt.Chart(movies).mark_circle().add_selection(
    selection
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(selection, alt.value(0.75), alt.value(0.05))
)

alt.Chart(...)

_Fun facts: The PG-13 rating didn't exist when the movies [Jaws](https://www.imdb.com/title/tt0073195/) and [Jaws 2](https://www.imdb.com/title/tt0077766/) were released. The first film to receive a PG-13 rating was 1984's [Red Dawn](https://www.imdb.com/title/tt0087985/)._

### Using Visualizations as Dynamic Queries

Though standard interface widgets show the _possible_ query parameter values, they do not visualize the _distribution_ of those values. We might also wish to use richer interactions, such as multi-value or interval selections, rather than input widgets that select only a single value at a time.

To address these issues, we can author additional charts to both visualize data and support dynamic queries. Let's add a histogram of the count of films per year and use an interval selection to dynamically highlight films over selected time periods.

*Interact with the year histogram to explore films from different time periods. Do you seen any evidence of [sampling bias](https://en.wikipedia.org/wiki/Sampling_bias) across the years? (How do year and critics' ratings relate?)*

_The years range from 1930 to 2040! Are future films in pre-production, or are there "off-by-one century" errors? Also, depending on which time zone you're in, you may see a small bump in either 1969 or 1970. Why might that be? (See the end of the notebook for an explanation!)_

In [11]:
brush = alt.selection_interval(
    encodings=['x'] # limit selection to x-axis (year) values
)

# dynamic query histogram
years = alt.Chart(movies).mark_bar().add_selection(
    brush
).encode(
    alt.X('year(Release_Date):T', title='Films by Release Year'),
    alt.Y('count():Q', title=None)
).properties(
    width=650,
    height=50
)

# scatter plot, modify opacity based on selection
ratings = alt.Chart(movies).mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(brush, alt.value(0.75), alt.value(0.05))
).properties(
    width=650,
    height=400
)

alt.vconcat(years, ratings).properties(spacing=5)

alt.VConcatChart(...)

The example above provides dynamic queries using a _linked selection_ between charts:

- We create an `interval` selection (`brush`), and set `encodings=['x']` to limit the selection to the x-axis only, resulting in a one-dimensional selection interval.
- We register `brush` with our histogram of films per year via `.add_selection(brush)`.
- We use `brush` in a conditional encoding to adjust the scatter plot `opacity`.

This interaction technique of selecting elements in one chart and seeing linked highlights in one or more other charts is known as [_brushing &amp; linking_](https://en.wikipedia.org/wiki/Brushing_and_linking).

## 6.4 Arrastamento &amp; Zoom

O gráfico de dispersão de classificações de filmes está um pouco sobrecarregado em alguns lugares, tornando difícil examinar pontos em regiões mais densas. Usando as técnicas de interação de _panning_ (arrastamento) e _zoomimg_ (zoom), podemos inspecionar as regiões densas mais de perto.

Vamos começar pensando em como podemos expressar arrastamento e zoom usando seleções do Altair. O que define a "área visível" de um gráfico? _Domínios de escala dos eixos!_

Podemos alterar os domínios das escalas para modificar o intervalo de dados visualizados. Para fazer isso de maneira interativa, podemos vincular uma seleção de `interval` (intervalos) aos domínios das escalas com o código `bind='scales'`. O resultado é que, em vez de um intervalo de seleção que podemos arrastar e ampliar, podemos arrastar e ampliar toda a área do gráfico!

_No gráfico abaixo, clique e arraste para deslocar (transladar) a visão ou use o scroll para dar zoom (ampliar) a visão. O que você pode descobrir sobre a precisão dos valores de rating fornecidos?_

In [ ]:
alt.Chart(movies).mark_circle().add_selection(
    alt.selection_interval(bind='scales')
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)), # use minExtent para estabilizar a posição do título do eixo
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
)

alt.Chart(...)

_Ao dar zoom, podemos ver que os valores de rating têm precisão limitada! Os ratings do Rotten Tomatoes são números inteiros, enquanto os do IMDB são truncadas para décimos. Como resultado, ocorre sobreposição de pontos mesmo quando damos zoom, com vários filmes compartilhando os mesmos valores de rating._

Ao ler o código acima, você pode notar o código `alt.Axis(minExtent=30)` no canal de codificação `y`. O parâmetro `minExtent` garante que um espaço mínimo seja reservado para as marcações e rótulos do eixo. Por que fazer isso? Quando fazemos o movimento de pan (arrastar) e zoom, os rótulos do eixo podem mudar e fazer com que a posição do título do eixo se desloque. Ao definir uma extensão mínima, podemos reduzir movimentos indesejados no gráfico. _Experimente mudar o valor de `minExtent`, por exemplo, configurando-o para zero, e então dê zoom para fora para ver o que acontece quando rótulos de eixo mais longos entram na visualização._

O Altair também inclui uma forma abreviada para adicionar arrastamento e zoom a um gráfico. Em vez de criar diretamente uma seleção, você pode chamar `.interactive()` para fazer o Altair gerar automaticamente uma seleção de intervalo limitada aos eixos do gráfico:

In [ ]:
alt.Chart(movies).mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)), # use minExtent para estabilizar a posição do título do eixo
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
).interactive()

alt.Chart(...)

Por padrão, as vinculações de escala para seleções incluem tanto os canais de codificação `x` quanto `y`. E se quisermos limitar o arrastamento e zoom a uma única dimensão? Podemos usar `encodings=['x']` para restringir a seleção ao canal `x` apenas:

In [ ]:
alt.Chart(movies).mark_circle().add_selection(
    alt.selection_interval(bind='scales', encodings=['x'])
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)), # use minExtent para estabilizar a posição do título do eixo
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
)

alt.Chart(...)

_Quando aplicamos zoom em apenas um dos eixos, o formato dos dados visualizados pode mudar, potencialmente afetando nossa percepção das relações nos dados. [Escolher uma proporção de aspecto adequada](http://vis.stanford.edu/papers/arclength-banking) é uma consideração importante no design de visualizações!_

## 6.5 Navegação: Visão Geral + Detalhe

Ao aplicar _panning_ (translação) e zoom, ajustamos diretamente a "janela de visualização" de um gráfico. A estratégia de navegação relacionada _visão geral + detalhe_ utiliza, em vez disso, uma exibição geral para mostrar _todos_ os dados, suportando seleções que arrastam e dão zoom em uma exibição de foco separada.

Abaixo, temos dois gráficos de área mostrando uma década de flutuações de preço do índice de ações S&amp;P 500. Inicialmente, ambos os gráficos exibem o mesmo intervalo de dados. _Clique e arraste no gráfico inferior de visão geral para atualizar a exibição de foco e examinar períodos específicos._

In [15]:
brush = alt.selection_interval(encodings=['x']);

base = alt.Chart().mark_area().encode(
    alt.X('date:T', title=None),
    alt.Y('price:Q')
).properties(
    width=700
)
  
alt.vconcat(
    base.encode(alt.X('date:T', title=None, scale=alt.Scale(domain=brush))),
    base.add_selection(brush).properties(height=60),
    data=sp500
)

alt.VConcatChart(...)

Ao contrário do caso anterior de arrastar e dar zoom, aqui não queremos vincular uma seleção diretamente às escalas de um único gráfico interativo. Em vez disso, queremos vincular a seleção ao domínio da escala em _outro_ gráfico. Para isso, atualizamos o canal de codificação `x` para o nosso gráfico de foco, definindo a propriedade `domain` da escala para referenciar nossa seleção `brush`. Se nenhum intervalo for definido (ou seja, se a seleção estiver vazia), o Altair ignora o _brush_ e usa os dados subjacentes para determinar o domínio. Quando um intervalo de _brush_ (intervalo de seleção) é criado, o Altair, contrariamente, o utiliza como o `domain` da escala para o gráfico de foco.

## 6.6 Detalhes sob Demanda

Uma vez que identificamos pontos de interesse dentro de uma visualização, frequentemente queremos saber mais sobre eles. _Detalhes sob demanda_ refere-se à indagação interativa por mais informações sobre os valores selecionados. _Tooltips_ são uma maneira útil de fornecer detalhes sob demanda. No entanto, os _tooltips_ tipicamente mostram informações para um ponto de dado por vez. Como podemos mostrar mais?

O gráfico de dispersão de ratings de filmes inclui vários _outliers_ (valores atípicos) potencialmente interessantes, onde as classificações do Rotten Tomatoes e do IMDB discordam. Vamos criar um gráfico que nos permita selecionar pontos interativamente e mostrar seus rótulos. Para acionar a consulta de filtro na interação de hover ou clique, usaremos o [operador de composição do Altair](https://altair-viz.github.io/user_guide/interactions.html#composing-multiple-selections) `|` ("ou").

_Passe o mouse sobre os pontos no gráfico de dispersão abaixo para ver um destaque e o título do rótulo. Clique com a tecla Shift para fazer anotações persistentes e visualizar vários rótulos ao mesmo tempo. Quais filmes são adorados pelos críticos do Rotten Tomatoes, mas não pelo público geral no IMDB (ou vice-versa)? Veja se consegue encontrar possíveis erros, onde dois filmes diferentes com o mesmo nome foram acidentalmente combinados!_

In [ ]:
hover = alt.selection_single(
    on='mouseover',  # selecionar ao passar o mouse
    nearest=True,    # selecionar o ponto mais próximo do cursor do mouse
    empty='none'     # seleção vazia não deve corresponder a nada
)

click = alt.selection_multi(
    empty='none' # seleção vazia não corresponde a pontos
)

# codificações do gráfico de dispersão compartilhadas por todas as marcas
plot = alt.Chart().mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q'
)
  
# base compartilhada para novas camadas
base = plot.transform_filter(
    hover | click # filtrar para pontos em qualquer seleção
)

# adicionar camadas de pontos do gráfico de dispersão, anotações de halo e rótulos de título
alt.layer(
    plot.add_selection(hover).add_selection(click),
    base.mark_point(size=100, stroke='firebrick', strokeWidth=1),
    base.mark_text(dx=4, dy=-8, align='right', stroke='white', strokeWidth=2).encode(text='Title:N'),
    base.mark_text(dx=4, dy=-8, align='right').encode(text='Title:N'),
    data=movies
).properties(
    width=600,
    height=450
)

alt.LayerChart(...)

O exemplo acima adiciona três novas camadas ao gráfico de dispersão: uma anotação circular, texto branco para fornecer um fundo legível e texto preto mostrando o título de um filme. Além disso, este exemplo usa duas seleções em conjunto:

1. Uma seleção simples (`hover`) que inclui `nearest=True` para selecionar automaticamente o ponto de dados mais próximo conforme o mouse se move.
2. Uma seleção múltipla (`click`) para criar seleções persistentes via shift-click.

Ambas as seleções incluem o conjunto `empty='none'` para indicar que nenhum ponto deve ser incluído se uma seleção estiver vazia. Essas seleções são então combinadas em um único predicado de filtro &mdash; o lógico _ou_ de `hover` e `click` &mdash; para incluir pontos que estejam em _qualquer uma_ das seleções. Usamos esse predicado a fim de filtrar as novas camadas para exibir anotações e rótulos apenas para os pontos selecionados.

Usando seleções e camadas, podemos realizar uma variedade de designs diferentes para detalhes sob demanda! Por exemplo, aqui está uma série temporal com escala logarítmica dos preços das ações de empresas de tecnologia, anotada com uma linha guia e rótulos para a data mais próxima ao cursor do mouse:

In [ ]:
# selecione um ponto para fornecer detalhes sob demanda
label = alt.selection_single(
    encodings=['x'], # limita a seleção ao valor do eixo x
    on='mouseover',  # seleciona nos eventos de mouseover (passar o mouse em cima)
    nearest=True,    # seleciona o ponto de dados mais próximo ao cursor
    empty='none'     # seleção vazia não inclui pontos de dados
)

# define nosso gráfico base de linha dos preços das ações
base = alt.Chart().mark_line().encode(
    alt.X('date:T'),
    alt.Y('price:Q', scale=alt.Scale(type='log')),
    alt.Color('symbol:N')
)

alt.layer(
    base, # gráfico base de linha
    
    # adiciona uma marca de linha vertical para servir como linha guia
    alt.Chart().mark_rule(color='#aaa').encode(
        x='date:T'
    ).transform_filter(label),
    
    # adiciona marcas de círculo para pontos de tempo selecionados, esconde pontos não selecionados
    base.mark_circle().encode(
        opacity=alt.condition(label, alt.value(1), alt.value(0))
    ).add_selection(label),

    # adiciona texto com contorno branco para fornecer um fundo legível para os rótulos
    base.mark_text(align='left', dx=5, dy=-5, stroke='white', strokeWidth=2).encode(
        text='price:Q'
    ).transform_filter(label),

    # adiciona rótulos de texto para os preços das ações
    base.mark_text(align='left', dx=5, dy=-5).encode(
        text='price:Q'
    ).transform_filter(label),
    
    data=stocks
).properties(
    width=700,
    height=400
)

alt.LayerChart(...)

_Pondo em ação o que aprendemos até agora: você pode modificar o gráfico de dispersão dos filmes acima (o com a consulta dinâmica sobre os anos) para incluir uma marca `rule` que mostre o rating IMDB (ou Rotten Tomatoes) médio para os dados contidos na seleção de ano `interval`?_

## Brushing &amp; Linking, Revisited

Earlier in this notebook we saw an example of _brushing &amp; linking_: using a dynamic query histogram to highlight points in a movie rating scatter plot. Here, we'll visit some additional examples involving linked selections.

Returning to the `cars` dataset, we can use the `repeat` operator to build a [scatter plot matrix (SPLOM)](https://en.wikipedia.org/wiki/Scatter_plot#Scatterplot_matrices) that shows associations between mileage, acceleration, and horsepower. We can define an `interval` selection and include it _within_ our repeated scatter plot specification to enable linked selections among all the plots.

_Click and drag in any of the plots below to perform brushing &amp; linking!_

In [18]:
brush = alt.selection_interval(
    resolve='global' # resolve all selections to a single global instance
)

alt.Chart(cars).mark_circle().add_selection(
    brush
).encode(
    alt.X(alt.repeat('column'), type='quantitative'),
    alt.Y(alt.repeat('row'), type='quantitative'),
    color=alt.condition(brush, 'Cylinders:O', alt.value('grey')),
    opacity=alt.condition(brush, alt.value(0.8), alt.value(0.1))
).properties(
    width=140,
    height=140
).repeat(
    column=['Acceleration', 'Horsepower', 'Miles_per_Gallon'],
    row=['Miles_per_Gallon', 'Horsepower', 'Acceleration']
)

alt.RepeatChart(...)

Note above the use of `resolve='global'` on the `interval` selection. The default setting of `'global'` indicates that across all plots only one brush can be active at a time. However, in some cases we might want to define brushes in multiple plots and combine the results. If we use `resolve='union'`, the selection will be the _union_ of all brushes: if a point resides within any brush it will be selected. Alternatively, if we use `resolve='intersect'`, the selection will consist of the _intersection_ of all brushes: only points that reside within all brushes will be selected.

_Try setting the `resolve` parameter to `'union'` and `'intersect'` and see how it changes the resulting selection logic._

### Cross-Filtering

The brushing &amp; linking examples we've looked at all use conditional encodings, for example to change opacity values in response to a selection. Another option is to use a selection defined in one view to _filter_ the content of another view.

Let's build a collection of histograms for the `flights` dataset: arrival `delay` (how early or late a flight arrives, in minutes), `distance` flown (in miles), and `time` of departure (hour of the day). We'll use the `repeat` operator to create the histograms, and add an `interval` selection for the `x` axis with brushes resolved via intersection.

In particular, each histogram will consist of two layers: a gray background layer and a blue foreground layer, with the foreground layer filtered by our intersection of brush selections. The result is a _cross-filtering_ interaction across the three charts!

_Drag out brush intervals in the charts below. As you select flights with longer or shorter arrival delays, how do the distance and time distributions respond?_

In [19]:
brush = alt.selection_interval(
    encodings=['x'],
    resolve='intersect'
);

hist = alt.Chart().mark_bar().encode(
    alt.X(alt.repeat('row'), type='quantitative',
        bin=alt.Bin(maxbins=100, minstep=1), # up to 100 bins
        axis=alt.Axis(format='d', titleAnchor='start') # integer format, left-aligned title
    ),
    alt.Y('count():Q', title=None) # no y-axis title
)
  
alt.layer(
    hist.add_selection(brush).encode(color=alt.value('lightgrey')),
    hist.transform_filter(brush)
).properties(
    width=900,
    height=100
).repeat(
    row=['delay', 'distance', 'time'],
    data=flights
).transform_calculate(
    delay='datum.delay < 180 ? datum.delay : 180', # clamp delays > 3 hours
    time='hours(datum.date) + minutes(datum.date) / 60' # fractional hours
).configure_view(
    stroke='transparent' # no outline
)

alt.RepeatChart(...)

_By cross-filtering you can observe that delayed flights are more likely to depart at later hours. This phenomenon is familiar to frequent fliers: a delay can propagate through the day, affecting subsequent travel by that plane. For the best odds of an on-time arrival, book an early flight!_

The combination of multiple views and interactive selections can enable valuable forms of multi-dimensional reasoning, turning even basic histograms into powerful input devices for asking questions of a dataset!

## Summary

For more information about the supported interaction options in Altair, please consult the [Altair interactive selection documentation](https://altair-viz.github.io/user_guide/interactions.html). For details about customizing event handlers, for example to compose multiple interaction techniques or support touch-based input on mobile devices, see the [Vega-Lite selection documentation](https://vega.github.io/vega-lite/docs/selection.html).

Interested in learning more?
- The _selection_ abstraction was introduced in the paper [Vega-Lite: A Grammar of Interactive Graphics](http://idl.cs.washington.edu/papers/vega-lite/), by Satyanarayan, Moritz, Wongsuphasawat, &amp; Heer.
- The PRIM-9 system (for projection, rotation, isolation, and masking in up to 9 dimensions) is one of the earliest interactive visualization tools, built in the early 1970s by Fisherkeller, Tukey, &amp; Friedman. [A retro demo video survives!](https://www.youtube.com/watch?v=B7XoW2qiFUA)
- The concept of brushing &amp; linking was crystallized by Becker, Cleveland, &amp; Wilks in their 1987 article [Dynamic Graphics for Data Analysis](https://scholar.google.com/scholar?cluster=14817303117298653693).
- For a comprehensive summary of interaction techniques for visualization, see [Interactive Dynamics for Visual Analysis](https://queue.acm.org/detail.cfm?id=2146416) by Heer &amp; Shneiderman.
- Finally, for a treatise on what makes interaction effective, read the classic [Direct Manipulation Interfaces](https://scholar.google.com/scholar?cluster=15702972136892195211) paper by Hutchins, Hollan, &amp; Norman.

#### Appendix: On The Representation of Time

Earlier we observed a small bump in the number of movies in either 1969 and 1970. Where does that bump come from? And why 1969 _or_ 1970? The answer stems from a combination of missing data and how your computer represents time. 

Internally, dates and times are represented relative to the [UNIX epoch](https://en.wikipedia.org/wiki/Unix_time), in which time "zero" corresponds to the stroke of midnight on January 1, 1970 in [UTC time](https://en.wikipedia.org/wiki/Coordinated_Universal_Time), which runs along the [prime meridian](https://en.wikipedia.org/wiki/Prime_meridian). It turns out there are a few movies with missing (`null`) release dates. Those `null` values get interpreted as time `0`, and thus map to January 1, 1970 in UTC time. If you live in the Americas &ndash; and thus in "earlier" time zones &ndash; this precise point in time corresponds to an earlier hour on December 31, 1969 in your local time zone. On the other hand, if you live near or east of the prime meridian, the date in your local time zone will be January 1, 1970.

The takeaway? Always be skeptical of your data, and be mindful that how data is represented (whether as date times, or floating point numbers, or latitudes and longitudes, _etc._) can sometimes lead to artifacts that impact analysis!